In [1]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
import numpy as np
import scipy

2022-11-23 21:03:59.869024: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-23 21:04:00.268254: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-23 21:04:00.331827: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-23 21:04:00.331858: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [2]:
trdata = ImageDataGenerator(
    rescale= 1./255,
    zoom_range= (0.73, 0.9),
    horizontal_flip= True,
    rotation_range= 10,
    width_shift_range= 0.10,
    fill_mode= 'constant',
    height_shift_range= 0.10,   
    brightness_range= (0.55, 0.9),
)
traindata = trdata.flow_from_directory(
    directory="/home/kmh/Desktop/newnewdataset/training",
    target_size=(224,224),
    batch_size= 32,  
    class_mode= 'categorical',
    shuffle= True, 
    seed= 1337
)

tsdata = ImageDataGenerator(
    rescale= 1./255,
)
testdata = tsdata.flow_from_directory(
    directory="/home/kmh/Desktop/newnewdataset/testing", 
    target_size=(224,224),
    batch_size= 32,  
    class_mode= 'categorical',
    shuffle= True, 
    seed= 1337
)

Found 3141 images belonging to 3 classes.
Found 90 images belonging to 3 classes.


In [3]:
num_classes = len(traindata.class_indices)  
train_labels = traindata.classes 
train_labels = to_categorical(train_labels, num_classes=num_classes)
valid_labels = testdata.classes 
valid_labels = to_categorical(valid_labels, num_classes=num_classes)

nb_train_samples = len(traindata.filenames)  
nb_valid_samples = len(testdata.filenames)

In [4]:
vgg16 = VGG16(include_top= False, input_shape= (224, 224, 3), weights= 'imagenet')
vgg16.summary()

2022-11-23 21:04:14.234684: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-23 21:04:14.234710: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-23 21:04:14.234735: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (kmh): /proc/driver/nvidia/version does not exist
2022-11-23 21:04:14.235495: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


58889256/58889256 [==============================] - 18s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                             

In [5]:

model = Sequential()

for layer in vgg16.layers:
    model.add(layer)

for layer in model.layers:
    layer.trainable= False

model.add(Flatten(input_shape= (56, 56, 256)))
model.add(Dropout(0.2))
model.add(Dense(3,activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [6]:
model.compile(optimizer= keras.optimizers.Adam(lr= 0.0001), loss= 'categorical_crossentropy', metrics= ['accuracy'])

/home/kmh/Desktop/SEM7/DL/proj-test/env/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [10]:
checkpoint = ModelCheckpoint(
#  'baseline_model.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    mode='auto',
    save_weights_only=False,
    period=1
)

earlystop = EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=3,
    verbose=1,
    mode='auto'
)

# csvlogger = CSVLogger(
#     filename= "baseline_training_csv.log",
#     separator = ",",
#     append = False
# )

reduceLR = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=3,
    verbose=1, 
    mode='auto'
)

# callbacks = [checkpoint, earlystop, csvlogger,reduceLR]
callbacks = [checkpoint, earlystop, reduceLR]

In [11]:
history = model.fit(
    traindata, 
    epochs = 30,
    steps_per_epoch = nb_train_samples//32,
    validation_data = testdata, 
    validation_steps = nb_valid_samples//32,
    verbose = 2,
    callbacks = callbacks,
    shuffle = True
)

Epoch 1/30

Epoch 1: val_loss improved from inf to 0.62100, saving model to baseline_model.h5
98/98 - 390s - loss: 0.8379 - accuracy: 0.6156 - val_loss: 0.6210 - val_accuracy: 0.6875 - lr: 1.0000e-04 - 390s/epoch - 4s/step
Epoch 2/30

Epoch 2: val_loss improved from 0.62100 to 0.45607, saving model to baseline_model.h5
98/98 - 411s - loss: 0.5288 - accuracy: 0.8112 - val_loss: 0.4561 - val_accuracy: 0.8438 - lr: 1.0000e-04 - 411s/epoch - 4s/step
Epoch 3/30

Epoch 3: val_loss improved from 0.45607 to 0.37638, saving model to baseline_model.h5
98/98 - 408s - loss: 0.4099 - accuracy: 0.8678 - val_loss: 0.3764 - val_accuracy: 0.8594 - lr: 1.0000e-04 - 408s/epoch - 4s/step
Epoch 4/30

Epoch 4: val_loss did not improve from 0.37638
98/98 - 410s - loss: 0.3490 - accuracy: 0.8903 - val_loss: 0.4308 - val_accuracy: 0.7969 - lr: 1.0000e-04 - 410s/epoch - 4s/step
Epoch 5/30


KeyboardInterrupt: 